## Definitions

In [ ]:
import pandas as pd

inputDataset="5c8a6dd041819c1518b4a7d8" # Adobe Analytics: Demo Environment postValues
outputDataset="5de54156338f2518a7c09577" # Recommendations Input Dataset For Lab

item_id = "_experience.analytics.customDimensions.eVars.eVar13"
interactionType = "_experience.analytics.customDimensions.eVars.eVar15"
user_id = "_experience.analytics.customDimensions.eVars.eVar1"
timestamp = "timestamp"
tenant_id = "_experienceplatform"

client_context = PLATFORM_SDK_CLIENT_CONTEXT

## Load the purchases data

In [ ]:
from platform_sdk.dataset_reader import DatasetReader

dataset_reader = DatasetReader(client_context, inputDataset)
df = dataset_reader.limit(50000).read()

## Filtering

In [ ]:
# drop nulls
df = df.dropna(subset=[user_id, item_id, interactionType])


## Split items into individual records

In [ ]:
# vectorized (no loops) solution for splitting in pandas
# source: https://stackoverflow.com/a/48120674
def split_df(dataframe, col_name, sep):
    orig_col_index = dataframe.columns.tolist().index(col_name)
    orig_index_name = dataframe.index.name
    orig_columns = dataframe.columns
    dataframe = dataframe.reset_index()
    index_col_name = (set(dataframe.columns) - set(orig_columns)).pop()
    df_split = pd.DataFrame(
        pd.DataFrame(dataframe[col_name].str.split(sep).tolist())
        .stack().reset_index(level=1, drop=1), columns=[col_name])
    df = dataframe.drop(col_name, axis=1)
    df = pd.merge(df, df_split, left_index=True, right_index=True, how='inner')
    df = df.set_index(index_col_name)
    df.index.name = orig_index_name

    return df

df2 = split_df(df, item_id, "\|\|")

# Data Prep for saving back to platform

In [ ]:
filtered_column_list = [item_id, user_id, interactionType, timestamp]

df2 = df2[filtered_column_list]


df2.rename(columns={
    item_id: tenant_id + ".itemId",
    user_id: tenant_id + ".userId",
    interactionType: tenant_id + ".interactionType"
}, inplace=True)
df2[tenant_id +'.timestamp'] = df2['timestamp']


# Write new dataframe to platform

In [ ]:
from platform_sdk.models import Dataset
from platform_sdk.dataset_writer import DatasetWriter
dataset = Dataset(PLATFORM_SDK_CLIENT_CONTEXT).get_by_id(dataset_id=outputDataset)
dataset_writer = DatasetWriter(PLATFORM_SDK_CLIENT_CONTEXT, dataset)
write_tracker = dataset_writer.write(df2, file_format='json')